<a href="https://colab.research.google.com/github/Roberto-Lins/Projeto-de-avalia-o-Aula-4-/blob/main/Imers%C3%A3o_IA_Alura%2Bgoogle_Aula5_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q -U google-generativeai

In [3]:
#importações e configurações iniciais
import numpy as np
import pandas as pd
import google.generativeai as genai
from google.colab import userdata

api_key = userdata.get('SECRET_KEY')
genai.configure(api_key=api_key)

Listar os modelos disponíveis

In [4]:
for m in genai.list_models():
  #if 'generateContent' in m.supported_generation_methods:
  if 'embedContent' in m.supported_generation_methods:
   print(m.name)


models/embedding-001
models/text-embedding-004


In [5]:
#Exemplo de embedding
title = "A proxima geração de IA para desenvolvedores e Google Workspace"
sample_text = ("Título: A proxima geração de IA para desenvolvedores e Google Workspace"
  "\n"
  "Artigo completo:\n"
  "\n"
  "Gemini API & Google AI Studio: Uma maneira acessível de explorar e criar protótipos com aplicações de IA  generativa")
embeddings = genai.embed_content(model="models/embedding-001",
                                 content=sample_text,
                                 title=title,
                                 task_type="RETRIEVAL_DOCUMENT")
print(embeddings)
#mostra qual o valor de cada um dos embedd
#Quanto que esta ali o calculo desas palavras quando for pensar naquele plano, quanto que  elas está posicionadas para ter idéia do contexto


{'embedding': [0.05570889, -0.04546477, -0.017137753, 0.023663742, 0.06300907, -0.00391305, -0.023738578, -0.024083544, 0.06077608, 0.05759515, 0.012168291, 0.010391565, -0.048853323, -0.021484928, 0.0049693016, -0.020707563, 0.021962965, -0.0037535971, -0.036316827, 0.0007780102, 0.012374759, 0.00909646, -0.032135013, -0.07809225, -0.012735231, 0.023634695, 0.019083416, -0.036256284, -0.04283967, 0.031098668, -0.05213085, 0.04884218, -0.03416316, 0.0149478605, -0.032491278, -0.04319148, -0.019442739, -0.04363818, -0.00028892086, -0.00323151, 0.011745416, -0.084749505, -0.014466227, 0.01718401, -0.010397984, -0.019354232, 0.04473902, 0.048952736, 0.018922858, -0.05660753, 0.03137323, 0.022969576, 0.06893833, -0.04348902, 0.0013239338, -0.01002518, 0.023768384, -0.025883967, 0.022691492, 0.0061018793, -0.004653078, 0.027330901, -0.014397483, 0.055834975, 0.026456969, -0.05691162, -0.044046108, 0.000910827, 0.010299652, 0.037812408, 0.0017864234, 0.0059042554, 0.058170963, -0.06376226, -

In [6]:
#Listagem de documentos que serão buscados
DOCUMENT1 = {
    "Titulo": "Operação do sistema de controle climático",
    "Conteudo": "O Googlecar rem um sistema de controle climático que permite ajustar a temperatura e o fluxo de ar no carro. Para operar o sistema "}

DOCUMENT2 = {
    "Titulo": "Touchscreen",
    "Conteudo": "O seu Googlecar tem uma grande tela sensível ao toque que  fornece acesso a una variedade de recursos, incluindo navegação,"}

DOCUMENT3 = {
    "Titulo": "Mudança de marchas",
    "Conteudo": "Seu Googlecar tem uma transmissão automatica. Para trocar as marchas, bastas mover a alavanca de câmbio para a posição desejada"}

Documents = [DOCUMENT1,DOCUMENT2, DOCUMENT3]
#Lista que vaai juntar os três documentos.
#Importante lembrar que a estrutura [Titulo, Conteúdo], porque o modelo de embedding neste modelo exige quen tenha o conteúdo e um título até mesmo porque vamos estruturar,
#Em um dataframe para poder ser consumido no próximo modelo.


In [7]:
df = pd.DataFrame(Documents)
df.columns = ["Titulo", "Conteudo" ]
df

,Titulo,Conteudo
0,Operação do sistema de controle climático,O Googlecar rem um sistema de controle climáti...
1,Touchscreen,O seu Googlecar tem uma grande tela sensível a...
2,Mudança de marchas,Seu Googlecar tem uma transmissão automatica. ...


In [8]:
model = "models/embedding-001"

In [9]:
def embed_fn(title, text):
  return genai.embed_content(model=model,
                                 content=sample_text,
                                 title=title,
                                 task_type="RETRIEVAL_DOCUMENT")["embedding"]

In [10]:
df["Embeddings"] = df.apply(lambda row: embed_fn(row["Titulo"],row["Conteudo"]),axis=1)
df

,Titulo,Conteudo,Embeddings
0,Operação do sistema de controle climático,O Googlecar rem um sistema de controle climáti...,"[0.03686256, -0.041701846, -0.040529992, 0.049..."
1,Touchscreen,O seu Googlecar tem uma grande tela sensível a...,"[0.05548125, -0.0648793, -0.024267202, 0.00383..."
2,Mudança de marchas,Seu Googlecar tem uma transmissão automatica. ...,"[0.05702409, -0.033200473, -0.012747205, 0.027..."


In [15]:
def gerar_e_buscar_consulta(consulta, base, model):
 embedding_da_consulta = genai.embed_content(model=model,
                                             content=consulta,
                                             task_type="RETRIEVAL_QUERY")["embedding"]

 produtos_escalares = np.dot(np.stack(df["Embeddings"]), embedding_da_consulta)
 indice =  np.argmax(produtos_escalares)
 return df.iloc[indice]["Conteudo"]

In [17]:
consulta = "Como faço para trocar marchas em um carro do Google?"
trecho = gerar_e_buscar_consulta(consulta, df, model)
print(trecho)

Seu Googlecar tem uma transmissão automatica. Para trocar as marchas, bastas mover a alavanca de câmbio para a posição desejada


In [19]:
generation_config ={
    "candidate_count": 1,
    "temperature": 0.5,
    }

In [20]:
prompt = f"Reescreva esse texto de uma forma mais descontraída, sem adicionar informações que não façam parte do texto: {trecho}"

model_2 = genai.GenerativeModel("gemini-1.0-pro",
                                generation_config=generation_config)

response = model_2.generate_content(prompt)
print(response.text)
#Obs. a descontração da resposta da IA depende do nível da temperatura ("temperature": 0.5,)

Se liga, seu carango do Google tem uma caixa automática maneirinha! Pra trocar as marchas, é moleza: só dá um toque na alavanca e pronto!
